# 05 - Model Training

In this notebook, you'll train your video similarity model, track experiments, and evaluate results.

## Learning Objectives
- Train a deep learning model for video similarity
- Integrate experiment tracking with Weights & Biases (wandb)
- Evaluate model performance with relevant metrics
- Reflect on training choices and results
- **Complete 4 hands-on exercises** for practical training and analysis

---

In [ ]:
# Imports
import sys, os
from pathlib import Path
project_root = Path.cwd().parent
sys.path.append(str(project_root))
import torch
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import yaml
from utils.data_utils import VideoDataset
from utils.model_utils import VideoSiameseNetwork, VideoTripletNetwork
from utils.training_utils import ContrastiveLoss, TripletLoss
from torch.utils.data import DataLoader
import seaborn as sns
plt.style.use('default')
sns.set_palette("husl")
print("Libraries imported successfully!")

## 1. Experiment Setup

Let's load the config, prepare data, and initialize wandb.

In [ ]:
# Load config
config_path = project_root / 'configs' / 'default_config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Initialize wandb
wandb.init(project='video-similarity', config=config)
print("wandb initialized!")

In [ ]:
# Prepare data
data_dir = project_root / 'data' / 'videos'
metadata_file = data_dir / 'sample_metadata.csv'
pairs_file = data_dir / 'similarity_pairs.csv'
metadata = pd.read_csv(metadata_file)
pairs = pd.read_csv(pairs_file)
from sklearn.model_selection import train_test_split
train_pairs, val_pairs = train_test_split(
    pairs, test_size=0.2, random_state=42, stratify=pairs['similarity']
)
train_dataset = VideoDataset(
    pairs=train_pairs, video_dir=data_dir,
    max_frames=config['data']['max_frames'],
    image_size=config['data']['image_size'], is_training=True)
val_dataset = VideoDataset(
    pairs=val_pairs, video_dir=data_dir,
    max_frames=config['data']['max_frames'], image_size=config['data']['image_size'], is_training=False)
train_loader = DataLoader(train_dataset, batch_size=config['training']['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config['training']['batch_size'], shuffle=False, num_workers=2)
print(f'Train/Val samples: {len(train_dataset)}/{len(val_dataset)}')

## 🎯 EXERCISE 1: Custom Experiment Tracking

**Task**: Set up your own wandb project and experiment name.

- Change the wandb project name to something unique.
- Add tags and notes describing your experiment.
- Log a custom config parameter (e.g., 'student_name').

**Your code here:**

In [ ]:
# TODO: Customize your wandb experiment setup here


## 2. Model, Loss, and Optimizer Setup

Let's set up the model, loss function, and optimizer.

In [ ]:
if config['model']['architecture'] == 'siamese':
    model = VideoSiameseNetwork(
        feature_dim=config['model']['feature_dim'],
    embedding_dim=config['model']['embedding_dim'])
    loss_fn = ContrastiveLoss(margin=config['training']['margin'])
elif config['model']['architecture'] == 'triplet':
    model = VideoTripletNetwork(
        feature_dim=config['model']['feature_dim'],
    embedding_dim=config['model']['embedding_dim'])
    loss_fn = TripletLoss(margin=config['training']['margin'])
else:
    raise ValueError('Unknown architecture')
optimizer = torch.optim.Adam(model.parameters(), lr=config['training']['learning_rate'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f'Model: {type(model).__name__}, Loss: {type(loss_fn).__name__}, Optimizer: Adam')

## 🎯 EXERCISE 2: Model Customization

**Task**: Modify the model architecture or optimizer.

- Change the embedding dimension or add a dropout layer.
- Try a different optimizer (e.g., SGD, RMSprop).
- Log your changes to wandb.

**Your code here:**

In [ ]:
# TODO: Customize your model or optimizer here


## 3. Training Loop

Let's implement the training and validation loop with wandb logging.

In [ ]:
def train_one_epoch(model, loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for batch in tqdm(loader, desc='Train'):
        video1, video2, labels = batch
        video1, video2, labels = video1.to(device), video2.to(device), labels.to(device)
        optimizer.zero_grad()
        preds = model(video1, video2)
        loss = loss_fn(preds, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def validate_one_epoch(model, loader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in tqdm(loader, desc='Val'):
            video1, video2, labels = batch
            video1, video2, labels = video1.to(device), video2.to(device), labels.to(device)
            preds = model(video1, video2)
            loss = loss_fn(preds, labels.float())
            total_loss += loss.item()
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    return total_loss / len(loader), np.concatenate(all_preds), np.concatenate(all_labels)

## 🎯 EXERCISE 3: Training Enhancements

**Task**: Add at least one training enhancement.

- Add gradient clipping, early stopping, or learning rate scheduling.
- Log additional metrics to wandb (e.g., accuracy, ROC AUC).
- Visualize training curves.

**Your code here:**

In [ ]:
# TODO: Add training enhancements here


## 4. Run Training

Let's run the training loop and log results to wandb.

In [ ]:
n_epochs = config['training']['epochs']
for epoch in range(n_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn, device)
    val_loss, val_preds, val_labels = validate_one_epoch(model, val_loader, loss_fn, device)
    wandb.log({'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss})
    print(f'Epoch {epoch+1}/{n_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}')

## 🎯 EXERCISE 4: Results Reflection

**Task**: Reflect on your training results.

- Analyze your wandb logs and training curves.
- What worked well? What would you change?
- How did your model architecture and training choices affect results?
- Write a short summary of your findings.

**Your reflection here (markdown):**

# Summary

In this notebook, you:
- Set up and tracked experiments with wandb
- Trained a video similarity model
- Evaluated and reflected on your results
- Practiced hands-on model training and experiment analysis

**Next:** Try the final deliverable script to design and test your own architectures!